In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from typing import List
import numpy as np

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

In [4]:
from transformers import AutoModelForSequenceClassification


model_score = "microsoft/DialogRPT-updown"   
tokenizer_rpt = AutoTokenizer.from_pretrained(model_score)
model_rpt = AutoModelForSequenceClassification.from_pretrained(model_score)

In [5]:
def get_candidates(user_input:str,
                   context: List[str] = []) -> List[str]:
    """Your code here"""
    
    user_input = tokenizer.encode(user_input, tokenizer.eos_token, return_tensors='pt')
    
    beam_outputs = model.generate(
        user_input, 
        max_length=60, 
        num_beams=5, 
        no_repeat_ngram_size=2, 
        num_return_sequences=5, 
        early_stopping=True
    )
    beam_output = []
    for step in range(5):
        m = format(tokenizer.decode(beam_outputs[:, user_input.shape[-1]:][step], skip_special_tokens=True))
        beam_output.append(m)
    
    return beam_output, beam_outputs 

In [ ]:
def get_scores(context:List[str],
               user_input:str,
               candidates: List[str]) -> List[float]:
    
    """Your code here"""
    
    result = model_rpt(context, return_dict=True)
    candidate_scores = torch.sigmoid(result.logits)

    
    return candidate_scores

In [ ]:
def chat():
    #init the chat with 0 context
    context = []
    
    while True:
        
        #get user input
        user_input = input('user > ')
        
        candidates, output = get_candidates(user_input)
        print(candidates)
        print(output)
        
        scores = get_scores(output, user_input, candidates)
        
        print(scores)
        
        response = candidates[torch.argmax(scores)]

        print('Bot >', response)
        
        context.extend([user_input,response])
        
        print(context)

In [10]:
chat()

user > hey


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Hey! :D', 'Hiya!', 'Hey you', 'Hiya', 'hi']
tensor([[20342, 50256, 10814,  5145,  1058,    35, 50256],
        [20342, 50256, 17250,  3972,  5145, 50256, 50256],
        [20342, 50256, 10814,   345, 50256, 50256, 50256],
        [20342, 50256, 17250,  3972, 50256, 50256, 50256],
        [20342, 50256,  5303, 50256, 50256, 50256, 50256]])
tensor([[0.1823],
        [0.1960],
        [0.1952],
        [0.1852],
        [0.1587]], grad_fn=<SigmoidBackward>)
Bot > Hiya!
['hey', 'Hiya!']


KeyboardInterrupt: Interrupted by user